# Deep Learning for NLP - Project

RULES:

* Do not create any additional cell

* Fill in the blanks

* All cells should be runnable (modulo trivial compatibility bugs that we'd fix)

* 4 / 20 points will be allocated to the clarity of your code

* Efficient code will have a bonus

DELIVERABLE:

* this notebook
* the predictions of the SST test set

DO NOT INCLUDE THE DATASETS IN THE DELIVERABLE..

In [ ]:
import io
import os
import numpy as np
import scipy

In [ ]:
PATH_TO_DATA = "../data/" # On avait "../../data/" dans le fichier fourni

# 1) Monolingual (English) word embeddings 

In [ ]:
class Word2vec():
    def __init__(self, fname, nmax=100000):
        self.load_wordvec(fname, nmax)
        self.word2id = dict.fromkeys(self.word2vec.keys())
        self.id2word = {v: k for k, v in self.word2id.items()}
        self.embeddings = np.array(self.word2vec.values())
    
    def load_wordvec(self, fname, nmax):
        self.word2vec = {}
        with io.open(fname, encoding='utf-8') as f:
            next(f)
            for i, line in enumerate(f):
                word, vec = line.split(' ', 1)
                self.word2vec[word] = np.fromstring(vec, sep=' ')
                if i == (nmax - 1):
                    break
        print('Loaded %s pretrained word vectors' % (len(self.word2vec)))

    def most_similar(self, w, K=5):
        # K most similar words: self.score  -  np.argsort
        similarWords = np.array([[key,self.score(w, key)]for key in self.word2vec.keys() if w!=key])
        similarWords = similarWords[np.argsort(similarWords[:,1])]
        similarWords = similarWords[-K:,0]
        return(np.flipud(similarWords))

    def score(self, w1, w2):
        # cosine similarity: np.dot  -  np.linalg.norm
        vec1 = self.word2vec[w1]
        vec2 = self.word2vec[w2]
        score = np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
        return(round(score,4))

In [5]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=100000)

# You will be evaluated on the output of the following:
for w1, w2 in zip(('cat', 'dog', 'dogs', 'paris', 'germany'), ('dog', 'pet', 'cats', 'france', 'berlin')):
    print(w1, w2, w2v.score(w1, w2))
for w1 in ['cat', 'dog', 'dogs', 'paris', 'germany']:
    print(w2v.most_similar(w1))

Loaded 100000 pretrained word vectors
cat dog 0.6717
dog pet 0.6842
dogs cats 0.7074
paris france 0.7775
germany berlin 0.742
['cats' 'kitty' 'kitten' 'feline' 'kitties']
['dogs' 'puppy' 'Dog' 'doggie' 'canine']
['dog' 'pooches' 'Dogs' 'doggies' 'canines']
['france' 'Paris' 'london' 'berlin' 'tokyo']
['austria' 'europe' 'german' 'berlin' 'poland']


In [ ]:
class BoV():
    def __init__(self, w2v):
        self.w2v = w2v
    
    def encode(self, sentences, idf=False, sentence = False): #sentence est la phrase qu'on veut encoder c'est un string
        # takes a list of sentences, outputs a numpy array of sentence embeddings
        sentemb = [] #array of sentences embeddings
        listWords = []
        for sent in sentences:
            sent = sent.split()
            for word in sent:
                listWords.append(word)
        listWords = list(set(listWords)) #List which contains all words of the document
        #We enter in this if only if we want to encode only one string
        if sentence!= False:
            sent = sentence.split()
            l = np.zeros(len(listWords)); #list which contain occurence of each word in a sentence
            for word in enumerate(listWords):
                if idf is False:
                    # mean of word vectors
                    for wordSentence in sent:
                        if wordSentence == word[1]:
                            l[word[0]] +=1
                else:
                    # idf-weighted mean of word vectors
                    for wordSentence in sent:
                        if wordSentence == word[1]:
                            l[word[0]] +=idf[wordSentence]
            sentemb.append(l)
            return np.vstack(sentemb)
        #We enter in this boucle when we want to encode all the file
        for sent in sentences[:1000]: #We are interested in the first 100 sentences to limit the calcul
            sent = sent.split()
            l = np.zeros(len(listWords)); #list which contain occurence of each word in a sentence
            for word in enumerate(listWords):
                if idf is False:
                    # mean of word vectors
                    for wordSentence in sent:
                        if wordSentence == word[1]:
                            l[word[0]] +=1
                else:
                    # idf-weighted mean of word vectors
                    for wordSentence in sent:
                        if wordSentence == word[1]:
                            l[word[0]] +=idf[wordSentence]
            sentemb.append(l)
        return np.vstack(sentemb)

    def most_similar(self, s, sentences, idf=False, K=5):
        # get most similar sentences and **print** them
        sentencesVectors = self.encode(sentences, idf)
        query = self.encode(sentences, idf, s)[0]
        similarSentences = np.array([[indice,self.score(query, sentence)]for indice,sentence in enumerate(sentencesVectors)])
        similarSentences = similarSentences[np.argsort(similarSentences[:,1])]
        similarSentences = similarSentences[-K-1:-1,:]
        similarSentences = np.flipud(similarSentences)
        L = []
        print("The ", str(K), " sentences the most similar to '", s, "' are:")
        for i in similarSentences:  
            print(sentences[int(i[0])])
            L.append(sentences[int(i[0])])
        return(L)

    def score(self, s1, s2, idf=False):
        # cosine similarity: use   np.dot  and  np.linalg.norm
        score = np.dot(s1,s2)/(np.linalg.norm(s1)*np.linalg.norm(s2))
        return(round(score,4))
    
    def build_idf(self, sentences):
        # build the idf dictionary: associate each word to its idf value
        idf = {}
        for sent in sentences:
            for w in list(set(sent.split())):
                idf[w] = idf.get(w, 0) + 1
        for word in idf.keys():
            idf[word] = max(1, np.log10(len(sentences) / (idf[word])))
        return(idf)


In [6]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=5000)
s2v = BoV(w2v)

# Load sentences in "PATH_TO_DATA/sentences.txt"
sentences = []
with io.open(os.path.join(PATH_TO_DATA, 'sentences.txt'), 'r') as file:
    txt = file.read()
sentences = txt.split("\n")

# Build idf scores for each word
idf = s2v.build_idf(sentences)

# You will be evaluated on the output of the following:
s2v.most_similar('' if not sentences else sentences[10], sentences)  # BoV-mean
print(s2v.score('' if not sentences else s2v.encode(sentences, False, sentences[7])[0], '' if not sentences else s2v.encode(sentences, False, sentences[13])[0]))

s2v.most_similar('' if not sentences else sentences[10], sentences, idf)  # BoV-idf
print(s2v.score('' if not sentences else s2v.encode(sentences, idf, sentences[7])[0], '' if not sentences else s2v.encode(sentences, idf, sentences[13])[0], idf))

Loaded 5000 pretrained word vectors
The  5  sentences the most similar to ' 1 smiling african american boy .  ' are:
5 women and 1 man are smiling for the camera . 
a african teacher , smiling , while working with baby on back . 
2 kids holding hands and smiling . 
2 african adults and 8 african children looking at pictures on a table . 
1 man singing and 1 man playing a saxophone in a concert . 
0.0765
The  5  sentences the most similar to ' 1 smiling african american boy .  ' are:
1 man singing and 1 man playing a saxophone in a concert . 
5 women and 1 man are smiling for the camera . 
2 guys facing away from camera , 1 girl smiling at camera with blue shirt , 1 guy with a beverage with a jacket on . 
1 woman in a black jacket is drinking out of a bottle while others are smiling . 
1 man riding a bike through the country . 
0.0134


# 2) Multilingual (English-French) word embeddings

Let's consider a bilingual dictionary of size V_a (e.g French-English).

Let's define **X** and **Y** the **French** and **English** matrices.

They contain the embeddings associated to the words in the bilingual dictionary.

We want to find a **mapping W** that will project the source word space (e.g French) to the target word space (e.g English).

Procrustes : **W\* = argmin || W.X - Y ||  s.t  W^T.W = Id**
has a closed form solution:
**W = U.V^T  where  U.Sig.V^T = SVD(Y.X^T)**

In what follows, you are asked to: 

In [81]:
# 1 - Download and load 50k first vectors of
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec

# TYPE CODE HERE
frDictionnary = Word2vec(os.path.join(PATH_TO_DATA, 'wiki.fr.vec'), nmax=10000)
enDictionnary = Word2vec(os.path.join(PATH_TO_DATA, 'wiki.en.vec'), nmax=10000)

Loaded 10000 pretrained word vectors
Loaded 10000 pretrained word vectors


In [83]:
# 2 - Get words that appear in both vocabs (= identical character strings)
#     Use it to create the matrix X and Y (of aligned embeddings for these words)

# TYPE CODE HERE
commonWords = []
for i in frDictionnary.word2vec.keys():
    if i in enDictionnary.word2vec:
        commonWords.append(i)

X = [] 
Y = []

for word in commonWords:
    X.append(frDictionnary.word2vec[word])
    Y.append(enDictionnary.word2vec[word])

In [86]:
# 3 - Solve the Procrustes using the scipy package and: scipy.linalg.svd() and get the optimal W
#     Now W*French_vector is in the same space as English_vector

# TYPE CODE HERE
import scipy.linalg
M = np.dot(Y,np.transpose(X))
U, Sig, V = scipy.linalg.svd(M, full_matrices = True)
W = np.dot(U,np.transpose(V))

In [85]:
# 4 - After alignment with W, give examples of English nearest neighbors of some French words (and vice versa)
#     You will be evaluated on that part and the code above

# TYPE CODE HERE
Xpredict = np.dot(np.transpose(W),Y)
Ypredict = np.dot(W,X)

def score(vec1, vec2):
        score = np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
        return(round(score,4))

# paris indice 52, jean indice 55, cat indice 1819 dans la liste commonWords

L = {'paris' : 52, 'jean' : 55, 'cat' : 1819}

#Français -> Anglais

for key, value in L.items():
    similarWords = np.array([[word,score(vector, Ypredict[value])]for word, vector in enDictionnary.word2vec.items()])
    similarWords = similarWords[np.argsort(similarWords[:,1])]
    similarWords = np.flipud(similarWords[-5:,0])
    print("The five words the more similar to " + str(key) + " in english are:")
    for word in similarWords:
        print(word)

#Anglais -> Français

for key, value in L.items():
    similarWords = np.array([[word,score(vector, Xpredict[value])]for word, vector in frDictionnary.word2vec.items()])
    similarWords = similarWords[np.argsort(similarWords[:,1])]
    similarWords = np.flipud(similarWords[-5:,0])
    print("The five words the more similar to " + str(key) + " in french are:")
    for word in similarWords:
        print(word)

The five words the more similar to paris in english are:
moral
questions
answers
thoughts
judgement
The five words the more similar to jean in english are:
difficulty
scenes
legendary
norse
hank
The five words the more similar to cat in english are:
gmina
delaware
paragraphs
footnote
prose
The five words the more similar to paris in french are:
régent
mariée
héritier
détenu
décoré
The five words the more similar to jean in french are:
par
composé
associés
trad
vendu
The five words the more similar to cat in french are:
famille
enfants
johan
figurer
classer


If you want to dive deeper on this subject: https://github.com/facebookresearch/MUSE

# 3) Sentence classification with BoV and scikit-learn

In [11]:
# 1 - Load train/dev/test of Stanford Sentiment TreeBank (SST)
#     (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)

# TYPE CODE HERE
sentencesTrain = []
sentencesDev = []
sentencesTest = []
with io.open(os.path.join(PATH_TO_DATA, 'SST/stsa.fine.train'), 'r') as file:
    txtTrain = file.read()
sentencesTrain = txtTrain.split("\n")
with io.open(os.path.join(PATH_TO_DATA, 'SST/stsa.fine.dev'), 'r') as file:
    txtDev = file.read()
sentencesDev = txtDev.split("\n")
with io.open(os.path.join(PATH_TO_DATA, 'SST/stsa.fine.test.X'), 'r') as file:
    txtTest = file.read()
sentencesTest = txtTest.split("\n")

sentencesTrain = [[sentence[1:] , sentence[0]] for sentence in sentencesTrain if len(sentence) >= 1]
sentencesDev = [[sentence[1:] , sentence[0]] for sentence in sentencesDev if len(sentence) >= 1]

In [12]:
# 2 - Encode sentences with the BoV model above

# TYPE CODE HERE
sentences = np.concatenate((np.array(sentencesTrain)[:100,0], np.array(sentencesDev)[:100,0], np.array(sentencesTest)[:100]), axis = 0)
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=50000)
s2v = BoV(w2v)
idf = s2v.build_idf(sentences)
encodeTestSet = [s2v.encode(sentences,idf,sentence)[0] for sentence in sentencesTest]
encodeTrainSet = [s2v.encode(sentences,idf,sentence)[0] for sentence in np.array(sentencesTrain)[:,0]]
encodeDevSet = [s2v.encode(sentences,idf,sentence)[0] for sentence in np.array(sentencesDev)[:,0]]

Loaded 50000 pretrained word vectors


In [23]:
# 3 - Learn Logistic Regression on top of sentence embeddings using scikit-learn
#     (consider tuning the L2 regularization on the dev set)

#Comparer avec et sans idf
#Essayer avec tous le dataset
#Utiliser le pca
#Enlever les mots de deux lettre ou moins, Sert à rien

# TYPE CODE HERE
from sklearn.linear_model import LogisticRegression
X = encodeTrainSet
y = np.array(sentencesTrain)[:,1]
clf = LogisticRegression(C = 0.25, penalty = 'l2', solver='newton-cg', multi_class='multinomial', random_state = 0).fit(X, y)
print("Train accuracy " + str(clf.score(X, y)) + "Dev accuracy " + str(clf.score(encodeDevSet, np.array(sentencesDev)[:,1])))

Train accuracy 0.6178604868913857Dev accuracy 0.3678474114441417


In [38]:
# 4 - Produce 2210 predictions for the test set (in the same order). One line = one prediction (=0,1,2,3,4).
#     Attach the output file "logreg_bov_y_test_sst.txt" to your deliverable.
#     You will be evaluated on the results of the test set.

# TYPE CODE HERE
yTest = clf.predict(encodeTestSet)
fichier = open("logreg_bov_y_test_sst.txt", "a")
fichier.write(yTest[0])
for i in yTest[1:]:
    fichier.write("\n" + i)
fichier.close()

In [39]:
# BONUS!
# 5 - Try to improve performance with another classifier
#     Attach the output file "XXX_bov_y_test_sst.txt" to your deliverable (where XXX = the name of the classifier)

# TYPE CODE HERE
from sklearn import svm
bdt = svm.LinearSVC(multi_class='ovr', penalty='l2')
bdt.fit(X,y)
print("Train accuracy " + str(bdt.score(X, y)) + "Dev accuracy " + str(bdt.score(encodeDevSet, np.array(sentencesDev)[:,1])))

yTestSVM = bdt.predict(encodeTestSet)
fichier = open("SVM_bov_y_test_sst.txt", "a")
fichier.write(yTestSVM[0])
for i in yTestSVM[1:]:
    fichier.write("\n" + i)
fichier.close()

C:\Users\Hugo\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train accuracy 0.6405664794007491Dev accuracy 0.3460490463215259


# 4) Sentence classification with LSTMs in Keras

## 4.1 - Preprocessing

In [17]:
import keras

Using TensorFlow backend.


In [18]:
# 1 - Load train/dev/test sets of SST
PATH_TO_DATA = "../../data/"

# TYPE CODE HERE


In [19]:
# 2 - Transform text to integers using keras.preprocessing.text.one_hot function
#     https://keras.io/preprocessing/text/

# TYPE CODE HERE


**Padding input data**

Models in Keras (and elsewhere) take batches of sentences of the same length as input. It is because Deep Learning framework have been designed to handle well Tensors, which are particularly suited for fast computation on the GPU.

Since sentences have different sizes, we "pad" them. That is, we add dummy "padding" tokens so that they all have the same length.

The input to a Keras model thus has this size : (batchsize, maxseqlen) where maxseqlen is the maximum length of a sentence in the batch.

In [20]:
# 3 - Pad your sequences using keras.preprocessing.sequence.pad_sequences
#     https://keras.io/preprocessing/sequence/

# TYPE CODE HERE


## 4.2 - Design and train your model

In [21]:
# 4 - Design your encoder + classifier using keras.layers
#     In Keras, Torch and other deep learning framework, we create a "container" which is the Sequential() module.
#     Then we add components to this contained : the lookuptable, the LSTM, the classifier etc.
#     All of these components are contained in the Sequential() and are trained together.


# ADAPT CODE BELOW


from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation

embed_dim  = 32  # word embedding dimension
nhid       = 64  # number of hidden units in the LSTM
vocab_size = 0  # size of the vocabulary
n_classes  = 5

model = Sequential()
model.add(Embedding(vocab_size, embed_dim))
model.add(LSTM(nhid, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(n_classes, activation='sigmoid'))


C:\Users\Hugo\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, dropout=0.2, recurrent_dropout=0.2)`


In [22]:
# 5 - Define your loss/optimizer/metrics

# MODIFY CODE BELOW

loss_classif     =  '' # find the right loss for multi-class classification
optimizer        =  '' # find the right optimizer
metrics_classif  =  ['accuracy']

# Observe how easy (but blackboxed) this is in Keras
model.compile(loss=loss_classif,
              optimizer=optimizer,
              metrics=metrics_classif)
print(model.summary())

ValueError: Unknown optimizer: 

In [ ]:
# 6 - Train your model and find the best hyperparameters for your dev set
#     you will be evaluated on the quality of your predictions on the test set

# ADAPT CODE BELOW
bs = 64
n_epochs = 6

history = model.fit(x_train, y_train, batch_size=bs, nb_epoch=n_epochs, validation_data=(x_val, y_val))

In [ ]:
# 7 - Generate your predictions on the test set using model.predict(x_test)
#     https://keras.io/models/model/
#     Log your predictions in a file (one line = one integer: 0,1,2,3,4)
#     Attach the output file "logreg_lstm_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE


## 4.3 -- innovate !

In [ ]:
# 8 - Open question: find a model that is better on your dev set
#     (e.g: use a 1D ConvNet, use a better classifier, pretrain your lookup tables ..)
#     you will get point if the results on the test set are better: be careful of not overfitting your dev set too much..
#     Attach the output file "XXX_XXX_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE
